In [180]:
import pandas as pd
import numpy as np
import re
data = pd.read_csv("properati.csv")

## Patrones -> Precio

In [181]:
# Precios inmueble

# detecto el precio (antecedido por usd y NO precedido por m2)

patron_precio_inm = '(usd|u\$d|u\$s|us\$|dolares) ?(\d{2,}(\,|\.)\d{0,3}\.?\d{0,3}|(\d{2,}(\,|\.)\d{0,3}\,?\.?\d{0,3}\,?\.?\d{0,3})) ?(?!(.*m2|.*mc|.*metros))'
regex_precio_inm = re.compile(patron_precio_inm, flags = re.IGNORECASE)

# formateo el precio (00.000.000 -> conjunto de tres cifras)

patron_precio_inm_2 = '(\d\d\d?\.)(\d\d\d\.)?(\d\d\d)'
regex_precio_inm_2 = re.compile(patron_precio_inm_2, flags = re.IGNORECASE)

### Búsqueda de Precios

#### Precios en descripción

In [182]:
# precios en descripción

precios_descripcion_macheo = data.description.apply(lambda x: x if x is np.NaN else regex_precio_inm.search(x))
precios_descripcion = precios_descripcion_macheo\
        .apply(lambda x: x if x is None or x is np.NaN else x.group(2))

print('Macheos encontrados de precios en descripción:',precios_descripcion_macheo.notnull().sum())
print('Ejemplo:',precios_descripcion.iloc[55245])
precios_descripcion.isnull().value_counts()

Macheos encontrados de precios en descripción: 7034
Ejemplo: 750.000132


True     114186
False      7034
Name: description, dtype: int64

In [183]:
# precios formateados con regex

precios_descripcion_formateo_match = precios_descripcion\
                    .apply(lambda x: x if x is None or x is np.NaN else regex_precio_inm_2.search(x))

precios_descripcion_formateado = precios_descripcion_formateo_match\
                    .apply(lambda x: np.NaN if x is None or x is np.NaN else x.group()\
                        .replace(',','')\
                        .replace('.',''))\
                    .astype(float)

precios_descripcion_formateado.iloc[55245]

750000.0

In [184]:
# formateo con condicional (mayores a cinco cifras y menores a nueve cifras)

precios_descripcion_filtado = precios_descripcion_formateado.apply(lambda x: x if x > 20000 and x < 30000000 else np.NaN)

print('Macheos de precios filtrados en titulo:', precios_descripcion_filtado.notnull().sum())

precios_descripcion_filtado.sort_values(ascending=False).head(10)

Macheos de precios filtrados en titulo: 5853


91468     25000000.0
91229     14000000.0
74066       993140.0
73965       985320.0
113972      980000.0
74032       961860.0
73927       954040.0
71653       950000.0
71409       950000.0
73979       946220.0
Name: description, dtype: float64

In [185]:
precios_descripcion_filtado.iloc[55245]

750000.0

#### Precios en título

In [186]:
# precios en descripción

precios_tit_macheo = data.title.apply(lambda x: x if x is np.NaN else regex_precio_inm.search(x))
precios_tit = precios_tit_macheo\
        .apply(lambda x: x if x is None or x is np.NaN else x.group(2))

print('Macheos encontrados de precios en descripción:',precios_tit_macheo.notnull().sum())
print('Ejemplo:',precios_tit.iloc[1937])
precios_tit.isnull().value_counts()

Macheos encontrados de precios en descripción: 6592
Ejemplo: 123.456.789


True     114628
False      6592
Name: title, dtype: int64

In [187]:
# formateos con regex

precios_tit_formateo_match = precios_tit\
                    .apply(lambda x: x if x is None or x is np.NaN else regex_precio_inm_2.search(x))

precios_tit_formateado = precios_tit_formateo_match\
                    .apply(lambda x: np.NaN if x is None or x is np.NaN else x.group()\
                        .replace(',','')\
                        .replace('.',''))\
                    .astype(float)

# el siguiente nro debe ser filtrado
precios_tit_formateado.iloc[1937]

123456789.0

In [188]:
# formateo con condicional (mayores a cinco cifras y menores a nueve cifras)

precios_tit_filtado = precios_tit_formateado.apply(lambda x: x if x > 20000 and x < 30000000 else np.NaN)

print('Macheos de precios filtrados en titulo:', precios_tit_filtado.notnull().sum())

precios_tit_filtado.sort_values(ascending=False).head(10)

Macheos de precios filtrados en titulo: 6574


34638    14000000.0
2082     11800000.0
525      11000000.0
8303       999999.0
5082       998000.0
17486      990000.0
9858       990000.0
11673      990000.0
21401      990000.0
21474      980000.0
Name: title, dtype: float64

### Revision

In [189]:
data.title.iloc[34638]

'Casa 2079m² en Del Libertador, Av. 2200, Vicente López, Olivos, por U$S 14.000.000'

In [190]:
data.description.iloc[91468]

'Casa con Agua Corriente, Córdoba, Punilla, por U$S 25.000.000'

### Fusión

In [191]:
precios_descripcion_filtado.notnull().sum()

5853

In [192]:
precios_tit_filtado.notnull().sum()

6574

In [193]:
data.price.notnull().sum()

100810

#### Price + Precios de descripción

In [228]:
print('precios vacíos en columna price:',data.price.isnull().sum())
print('precios a rellenar en columna price desde description:', precios_descripcion_filtado.loc[data.price.isnull()].notnull().sum())

# imputacion de valores extraídos de desc
# data['price_desc'] = data['price']
# data['price_desc'].loc[data.price.isnull()] = precios_descripcion_filtado.loc[data.price.isnull()]

print('precios vacios tras relleno con precios de desc:',data.price_tit.isnull().sum(),'\n20410 -','424 =',20410 - 2383)

precios vacíos en columna price: 20410
precios a rellenar en columna price desde description: 424
precios vacios tras relleno con precios de desc: 18027 
20410 - 424 = 18027


#### Price + Precios de titulo

In [229]:
print('precios vacíos en columna price:',data.price.isnull().sum())
print('precios a rellenar en columna price desde title:', precios_tit_filtado.loc[data.price.isnull()].notnull().sum())

# imputacion de valores extraídos de tit
# data['price_tit'] = data['price']
# data['price_tit'].loc[data.price.isnull()] = precios_tit_filtado.loc[data.price.isnull()]

print('precios vacios tras relleno con precios de tit:',data.price_tit.isnull().sum(),'\n20410 -','2383 =',20410 - 2383)

precios vacíos en columna price: 20410
precios a rellenar en columna price desde title: 2383
precios vacios tras relleno con precios de tit: 18027 
20410 - 2383 = 18027


### + Patrones

In [230]:
# Metros inmueble
metros_inmueble_pat = '(\d{0,4}\,?\.?\d{0,4})? ?(m2|mc|metros|mts)'
metros_inmueble_reg = re.compile(metros_inmueble_pat, flags = re.IGNORECASE)

# Cochera
cochera_pat = '(?<!sin )(?<!no incluye )(?<!no posee )cochera(?! no incluye)(?! no incluida)'
cochera_reg = re.compile(cochera_pat, flags = re.IGNORECASE)

# Piscina/Pileta
piscina_pat = 'piscina|pileta(?! de baño)(?! de cocina)'
piscina_reg = re.compile(piscina_pat, flags = re.IGNORECASE)

# Ambientes
amb_pat = '(\d{1,4})(?= ambientes)(?= amb)'
amb_reg = re.compile(amb_pat, flags = re.IGNORECASE)